# Анализ данных и проверка гипотез в бизнесе проката самокатов


## Цели и задачи проекта 

Крупный сервис проката самокатов GoFast предоставил три датасета с данными о пользователях сервиса, их поездках за один год, а также стоимости услуг в подписках. Необходимо проанализировать демографию пользователей и особенности использования самокатов, а также определить возможную выгоду от распространения платной подписки на самокаты.

### Описание данных

Таблица с пользователями `users_go.csv`

- `user_id` — уникальный идентификатор пользователя.

- `name` — имя пользователя.

- `age` — возраст.

- `city` — город.

- `subscription_type` — тип подписки: `free`, `ultra`.

Таблица с поездками `rides_go.csv`

- `user_id` — уникальный идентификатор пользователя.

- `distance` — расстояние в метрах, которое пользователь проехал в текущей сессии.

- `duration` — продолжительность сессии в минутах, то есть время с того момента, как пользователь нажал кнопку «Начать поездку», до того, как он нажал кнопку «Завершить поездку».

- `date` — дата совершения поездки.

Таблица с подписками `subscriptions_go.csv`

- `subscription_type` — тип подписки.

- `minute_price` — стоимость одной минуты поездки по этой подписке.

- `start_ride_price` — стоимость начала поездки.

- `subscription_fee` — стоимость ежемесячного платежа.

## Загрузка библиотек

In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

In [ ]:
import scipy.stats as st

In [ ]:
# Уменьшение разрешения DPI для графиков 
plt.rcParams['figure.dpi'] = 72

## Загрузка данных

In [ ]:
# Выгружаем данные из датасета 
try:
    #пробуем загрузить 
    df_users_go = pd.read_csv('datasets/users_go.csv')
    display('Файл загружен') 
except:
    # если не получилось
    display('Ошибка при загрузке файла')    

In [ ]:
# Выгружаем данные из датасета 
try:
    #пробуем загрузить 
    df_rides_go = pd.read_csv('datasets/rides_go.csv')
    display('Файл загружен') 
except:
    # если не получилось
    display('Ошибка при загрузке файла')    

In [ ]:
# Выгружаем данные из датасета 
try:
    #пробуем загрузить 
    df_subscriptions_go = pd.read_csv('datasets/subscriptions_go.csv')
    display('Файл загружен') 
except:
    # если не получилось
    display('Ошибка при загрузке файла')    

Познакомимся с содержанием таблиц, выведем первые пять строк каждого датафрейма. 

In [ ]:
df_users_go.head(5)

In [ ]:
df_rides_go.head(5)

In [ ]:
df_subscriptions_go.head(5)

## Первичная предобработка

Определим количество строк в каждом из трёх датафреймов

In [ ]:
display(f'{df_users_go.shape[0]} {df_rides_go.shape[0]} {df_subscriptions_go.shape[0]}')

In [ ]:
# Определим типы данных в датфрейме df_rides_go
df_rides_go.dtypes

In [ ]:
# Приведем столбец date в датафрейме df_rides_go к типу даты
df_rides_go['date'] = pd.to_datetime(df_rides_go['date'])

На основе столбца date создадим новый столбец month, содержащий номер месяца. Это нужно для последующей группировки данных и анализа сезонных трендов.

In [ ]:
df_rides_go['month'] = df_rides_go['date'].dt.month

В датафрейме поездок `df_rides_go` округлим время поездки `duration` до целого числа и приведем эту колонку к целочисленному типу `int`. Этот шаг поможет далее правильно рассчитать прибыль, так как плата взимается только за целое число минут.

In [ ]:
#округляем
df_rides_go['duration'] = round(df_rides_go['duration'],0)

In [ ]:
# преобразовываем к int64
df_rides_go['duration'] = df_rides_go['duration'].astype('int64', errors='raise')

In [ ]:
# проверяем
df_rides_go.dtypes

## Заполнение пропусков и удаление дубликатов

В датафрейме пользователей df_users_go определим количество пропусков и дубликатов.

In [ ]:
display(f'{df_users_go.isna().sum().sum()} {df_users_go.duplicated().sum()}')

In [ ]:
#удаляем дубликаты
df_users_go = df_users_go.drop_duplicates(keep='first', inplace=False) 

In [ ]:
#количество строк после удаления дубликатов в df_users_go
df_users_go.shape[0]

## Исследовательский анализ данных (EDA)

Проведем разведочный анализ данных (Exploratory Data Analysis). Изучим и визуализируем информацию о географии и демографии сервиса, закономерности в дистанциях и длительности поездок.

### Количество пользователей по городам.

Пользуясь данными из датафрейма df_users_go, изучим количество пользователей в каждом из городов. 

In [ ]:
users_by_city_count = df_users_go['city'].value_counts().sort_values(ascending=False)

In [ ]:
display(users_by_city_count)

### Количество пользователей подписки

Выведем на экран количество пользователей для каждого типа подписки.

In [ ]:
subscription_type_count = df_users_go['subscription_type'].value_counts().sort_values(ascending=False)

In [ ]:
display(subscription_type_count)

In [ ]:
subscription_type_count.plot(
    kind= 'pie',
    title='Соотношение пользователей с подпиской и без подписки',
    autopct= '%.0f%%',
    ylabel= '',
    colors= ['red','green']
)

plt.show()

### Возраст подписчиков

Построим гистограмму возрастов `age` пользователей самокатов. Используем количество бинов, равное разности максимального и минимального значений возраста.

In [ ]:
n_bins = df_users_go['age'].max() - df_users_go['age'].min()

In [ ]:
df_users_go['age'].hist(bins = n_bins)
plt.title('Возраст пользователей')
plt.xlabel('Возраст')
plt.show()

### Доля несовершеннолетних пользователей

Рассчитаем долю несовершеннолетних (возрастом менее 18 лет) пользователей самокатов.

In [ ]:
users_under_18_ratio = int((df_users_go.loc[df_users_go['age'] < 18].shape[0] / df_users_go.shape[0])*100)
display(f'Доля несовершеннолетних пользователей самокатов составляет {users_under_18_ratio}%.')

### Характеристики длительности поездки

Длительность поездки является важной метрикой в работе сервиса проката самокатов. Если средняя длительность поездок будет слишком высокой, самокаты будут быстрее выходить из строя. Если слишком низкой, значит, клиентам что-то не нравится в сервисе. Используем методы описательной статистики.

In [ ]:
duration_mean = int(round(df_rides_go['duration'].mean(),0))
duration_std = int(df_rides_go['duration'].std())

duration_pct25 = int(df_rides_go['duration'].quantile(0.25))
duration_pct75 = int(df_rides_go['duration'].quantile(0.75))

In [ ]:
display(f'Средняя длительность поездки {duration_mean} минут со стандартным отклонением {duration_std}.\
 Основная часть поездок занимает от {duration_pct25} до {duration_pct75} минут.')

### Объединение данных

Объединим датафреймы с информацией о пользователях df_users_go и поездках df_rides_go.

In [ ]:
df = df_users_go.merge(df_rides_go, on = 'user_id', how = 'left')

Присоединим к полученному датафрейму df информацию о подписках из df_subscriptions_go

In [ ]:
df = df.merge(df_subscriptions_go, on = 'subscription_type', how='left')

In [ ]:
# Выводим первые строки датафрейма
display(df.head(5))

# Выводим количество строк и столбцов в объединённом датафрейме
n_rows, n_cols = df.shape
display(f'В полученном датафрейме {n_rows} строк и {n_cols} столбцов.')

Создадим два вспомогательных датафрейма на основе `df`: первый только для пользователей с подпиской `df_ultra` и второй только для пользователей без подписки `df_free`. Эти датафрейма нужны для изучении поведения пользователей с подпиской и без, а также при проверке продуктовых гипотез.

In [ ]:
df_ultra = df.loc[df['subscription_type'] == 'ultra']

In [ ]:
df_free = df.loc[df['subscription_type'] == 'free']

### Гистограмма длительности поездок для обоих групп

На одном графике построим гистограмму распределения длительности поездок `duration` для пользователей с подпиской и без.

In [ ]:
# Гистограмма длительности поездки для пользователей с подпиской и без
plt.figure(figsize=(15, 5))
df_free['duration'].hist(bins = 30, label='free', color='red')
df_ultra['duration'].hist(bins = 30, label='ultra', color='green')
plt.title('Гистограмма распределения длительности поездок')
plt.xlabel('Длительность поездки, мин.')
plt.legend()
plt.show()

In [ ]:
# Расчет и вывод на экран средней длительности поездки для пользователей с подпиской и без
mean_duration_free = int(round(df_free['duration'].mean(),0))
mean_duration_ultra = int(round(df_ultra['duration'].mean(),0))
display(f'Средняя длительность поездки для пользователей без подписки {mean_duration_free} мин,\
 а для пользователей с подпиской {mean_duration_ultra} мин')

### Подсчёт выручки на каждого пользователя

Сгруппируем данные по следующим столбцам: user_id, name, subscription_type, month

In [ ]:
df_gp = df.groupby(['user_id', 'name', 'subscription_type','month'], as_index=False)

In [ ]:
df_gp.head(10)

In [ ]:
# Сагрегируем данные, используем т.н. именованное агрегирование
df_agg = df_gp.agg(
    total_distance=('distance', 'sum'),
    total_duration=('duration','sum') ,
    rides_count=('duration', 'count') ,
    subscription_type=('subscription_type', 'first') ,
    minute_price=('minute_price', 'first'),
    start_ride_price=('start_ride_price', 'first') ,
    subscription_fee=('subscription_fee','first')
)

In [ ]:
df_agg.head()

Создадим функцию `calculate_monthly_revenue(row)` для расчёта месячной выручки по формуле:
`monthly_revenue` = `start_ride_price` * `rides_count` + `minute_price` * `total_duration` + `subscription_fee`

В качестве входных данных функция будет принимать одну строку `row` датафрейма. 

- `start_ride_price * rides_count` — выручка от начала каждой поездки.
- `minute_price * total_duration` — выручка за время использования.
- `subscription_fee` — фиксированная выручка от подписок.

In [ ]:
def calculate_monthly_revenue(row):
    monthly_revenue =  row['start_ride_price'] * row['rides_count'] + row['minute_price'] * row['total_duration'] + row['subscription_fee']           
    return monthly_revenue

Создадим новый столбец с месячной выручкой на пользователя monthly_revenue. Для этого применим функцию calculate_monthly_revenue(row) к каждой строке агрегированного датафрейма df_agg.

In [ ]:
df_agg['monthly_revenue'] = df_agg.apply(calculate_monthly_revenue, axis=1)

In [ ]:
df_agg.head()

### Пользователь с максимальной выручкой¶

Найдем пользователя с максимальной суммарной выручкой за весь период наблюдения и выведем на экран user_id, имя, месяц, количество поездок и выручку за каждый из месяцев.

In [ ]:
user_max = pd.Series(df_agg.groupby('user_id')['monthly_revenue'].sum().sort_values(ascending = False).head(1))

In [ ]:
user_max = pd.DataFrame({'user_id': user_max.index, 'monthly_revenue':user_max.values})

In [ ]:
user_max_id = user_max.iloc[0,0]

In [ ]:
df_agg[['user_id','name','month','rides_count','monthly_revenue']].loc[df_agg['user_id'] == user_max_id]

## Проверка гипотез

### Вспомогательная функция для интерпретации результатов

Напишем вспомогательную функцию `print_stattest_results(p_value, alpha)`, которая будет интерпретировать результаты статистического теста на основе p-value и заданного уровня значимости (α-уровня). Функция должна решать, следует ли принять альтернативную гипотезу или сохранить нулевую гипотезу.

У функции два параметра:
- `p_value` (тип `float`) — значение p-value, полученное в результате выполнения статистического теста.
- `alpha` (тип `float`, необязательный, по умолчанию `alpha = 0.05`) — уровень значимости статистического теста, который используется как порог для принятия решения об отклонении нулевой гипотезы.

Если мы принимаете альтернативную гипотезу, выведем сообщение:
`'Полученное значение p_value=<Введённое значение p_value> меньше критического уровня alpha=0.05. Принимаем альтернативную гипотезу.'`

Если мы не можете опровергнуть нулевую гипотезу, выведем сообщение: `'Полученное значение p_value=<Введённое значение p_value> больше критического уровня alpha=0.05. Опровергнуть нулевую гипотезу нельзя.'`

In [ ]:
def display_stattest_results(p_value:float, alpha:float = 0.05):
    if p_value < alpha:
        display(f'Полученное значение p_value={p_value} меньше критического уровня alpha={alpha}. Принимаем альтернативную гипотезу.') 
    else:
        display(f'Полученное значение p_value={p_value} больше критического уровня alpha={alpha}. Опровергнуть нулевую гипотезу нельзя.') 
    pass      

### Длительность для пользователей с подпиской и без

Важно понять, тратят ли пользователи с подпиской больше времени на поездки? Сформулируем нулевую и альтернативную гипотезы:
- Нулевая гипотеза (Н0): Среднее время поездки у пользователей с подпиской и без подписки одинаковое.
- Альтернативная гипотеза (Н1): Среднее время поездки у пользователей с подпиской больше, чем у пользователей без подписки.

Чтобы проверить эту гипотезу:
1. Используем неагрегированные данные из датафреймов `df_ultra` и `df_free`, созданные на одном из прошлых шагов.
2. Используем данные о продолжительности каждой поездки `duration` — отдельно для подписчиков и тех, у кого нет подписки.
3. Рассчитаем значение `p_value` для выбранной гипотезы, используя функции модуля `scipy.stats` и односторонний t-тест.
4. В качестве результата вызовем написанную функцию `display_stattest_results(p_value, alpha)`, передав ей рассчитанное значение `p_value`.
5. Дополнительно рассчитаем среднюю длительность поездки для тарифов `ultra` и `free`, округлив её до двух знаков после точки.

In [ ]:
ultra_duration = df_ultra['duration']
free_duration = df_free['duration']

results = st.ttest_ind(ultra_duration, free_duration, alternative='greater' ) 
p_value = results.pvalue

display_stattest_results(p_value)

In [ ]:
ultra_mean_duration = round(df_ultra['duration'].mean(),2)
free_mean_duration = round(df_free['duration'].mean(),2)

display(f'Средняя длительность поездки тарифа Ultra {ultra_mean_duration}')
display(f'Средняя длительность поездки тарифа Free {free_mean_duration}')

### Длительность поездки: больше или меньше критического значения

Проанализируем другую важную продуктовую гипотезу. Расстояние одной поездки в 3130 метров — оптимальное с точки зрения износа самоката. Можно ли сказать, что расстояние, которое проезжают пользователи с подпиской за одну поездку, меньше 3130 метров?

Сформулируем нулевую и альтернативную гипотезы:
- Нулевая гипотеза (Н0): Средняя дистанция поездки у пользователей с подпиской равна 3130 м.
- Альтернативная гипотеза (Н1): Средняя дистанция поездки у пользователей с подпиской больше 3130 м.

Чтобы проверить эту гипотезу:
1. Используем неагрегированные данные о каждой поездке пользователей с подпиской из датафрейма `df_ultra`.
2. Используем данные о дистанции каждой поездки `distance`.
3. Рассчитаем значение `p_value` для выбранной гипотезы, используя функции модуля `scipy.stats` и односторонний t-тест. 
4. В качестве результата вызовем написанную функцию `display_stattest_results(p_value, alpha)`, передав ей рассчитанное значение `p_value`.

In [ ]:
null_hypothesis = 3130
ultra_distance = df_ultra['distance']

results = st.ttest_1samp(ultra_distance, null_hypothesis, alternative='greater' )
p_value = results.pvalue

In [ ]:
display_stattest_results(p_value)

### Прибыль от пользователей с подпиской и без

Проверим гипотезу о том, что выручка от пользователей с подпиской выше, чем выручка от пользователей без подписки.

Сформулируем нулевую и альтернативную гипотезы:
- Нулевая гипотеза (Н0): Средняя месячная выручка у пользователей с подпиской и без подписки одинаковая.
- Альтернативная гипотеза (Н1): Средняя месячная выручка у пользователей с подпиской выше, чем у пользователей без подписки.

Чтобы проверить эту гипотезу:
1. Используем агрегированные данные из датафрейма `df_agg`, подготовленного ранее.
2. Используем исходные данные о месячной выручке от каждого пользователя — `monthly_revenue`.
3. Рассчитаем значение `p_value` для выбранной гипотезы, используя функции модуля `scipy.stats` и односторонний t-тест.
4. В качестве результата вызовем написанную функцию `display_stattest_results(p_value, alpha)`, передав ей рассчитанное значение `p_value`.
5. Дополнительно рассчитаем среднюю выручку для тарифов `ultra` и `free`, округлив её до целого.

In [ ]:
revenue_ultra = df_agg['monthly_revenue'].loc[df_agg['subscription_type'] == 'ultra' ]
revenue_free = df_agg['monthly_revenue'].loc[df_agg['subscription_type'] == 'free' ]

results = st.ttest_ind(revenue_ultra, revenue_free, alternative='greater' )
p_value = results.pvalue
display_stattest_results(p_value)

In [ ]:
mean_revenue_ultra = int(round(df_agg['monthly_revenue'].loc[df_agg['subscription_type'] == 'ultra' ].mean(),0))
mean_revenue_free = int(round(df_agg['monthly_revenue'].loc[df_agg['subscription_type'] == 'free' ].mean(),0))

display(f'Средняя выручка подписчиков Ultra {mean_revenue_ultra} руб')
display(f'Средняя выручка подписчиков Free {mean_revenue_free} руб')

### Проверка бизнес-гипотезы с использованием распределений

В компании возникла идея предлагать дополнительную скидку подписчикам, совершающим длительные поездки продолжительностью более 30 минут. Оценим долю таких поездок.


Ранее мы уже построили гистограмму распределения длительности поездок для выборки. Однако эти данные охватывают лишь часть пользователей всех самокатов, а нас интересуют возможные значения для всей генеральной совокупности. Учитывая, что у нас нет доступа ко всем данным о поездках, было решено смоделировать длительность поездки с помощью нормального распределения, используя в качестве параметров выборочное среднее и стандартное отклонение из доступных данных о поездках.

#### Расчёт выборочного среднего и стандартного отклонения

1. Расчитаем среднюю длительность поездки и сохраните в переменную `mu`.
2. Вычислим стандартное отклонение длительности `duration` и сохраним в переменную `sigma`. 
3. Зададим значение переменной `target_time`, равное `30`. Эта переменная будет использоваться для последующего вычисления вероятности.

In [ ]:
# Вычисляем среднее значение
mu = df_ultra['duration'].mean()

# Вычисляем стандартное отклонение
sigma = df_ultra['duration'].std()

# Задаём целевое время
target_time = 30

# Делаем вывод
display(f'Средняя длительность поездки {round(mu, 1)}, стандартное отклонение {round(sigma)}.')

#### Вычисление значения функции распределения в точке (CDF)

Если вычислить значение функции распределения в точке, это позволит узнать вероятность того, что случайная величина примет значение меньше заданного либо равное ему. Соответственно, если мы хотите ответить на вопрос о вероятности поездки более 30 минут, используем CDF.

1. Используем функцию `norm()` из библиотеки SciPy для создания нормального распределения с параметрами `mu` и `sigma`.
2. Применим метод `cdf()` к целевому времени `target_time` для получения вероятности того, что случайная величина будет меньше этого значения или равна ему. 

In [ ]:
# Вычисляем вероятность того, что случайная величина будет меньше указанного значения или равна ему

duration_norm_dist  = st.norm(mu,sigma)

prob = round(1 - duration_norm_dist.cdf(target_time),3) # Используем CDF для нахождения накопленной вероятности

display(f'Вероятность поездки более 30 минут {prob}')

#### Вероятность для интервала (CDF)

Коллеги посчитали, что процент пользователей, для которых будет показана скидка, недостаточно большой и вряд ли поможет в увеличении лояльности клиентов. Дополнительно проверим, какой процент пользователей совершает поездки в интервале от 20 до 30 минут. Возможно, именно для них стоит провести промоакцию?

Для этого:
1. Создадим переменные `low` и `high`, указывающие на начало и конец интересующего временного интервала. В этом случае они равны 20 и 30 минут.
2. Используем кумулятивную функцию распределения (CDF) для объекта `duration_norm_dist`, чтобы вычислить вероятность достижения верхней границы (`high`) и нижней границы (`low`).
3. Вычислим вероятность попадания в интервал

In [ ]:
# Определяем границы интервала
low = 20
high = 30

prob_low = duration_norm_dist.cdf(low)
prob_high = duration_norm_dist.cdf(high)

# Вычисляем вероятность попадания в интервал
prob_interval = round(prob_high - prob_low,  3)

# Выводим результат
display(f'Вероятность того, что пользователь совершит поездку длительностью от {low} до {high} минут: {prob_interval}')

#### Определение критической дистанции поездок (PPF)

Длительные поездки могут негативно сказываться на сроке службы самоката. В связи с этим принято решение установить критическую дистанцию, превышение которой будет сопровождаться дополнительной платой. Для этого необходимо определить расстояние, которое превышается только в 10% поездок (90-й процентиль).

Задача — смоделировать распределение длительности поездок, предполагая, что оно подчиняется нормальному закону, и рассчитать критическую дистанцию, ниже которой находится 90% всех поездок.

Для этого:
1. Рассчитаем среднюю дистанцию поездки для всех пользователей из датафрейма df (с подпиской и без) и сохраните в переменную mu.
2. Вычислим стандартное отклонение дистанции поездки distance и сохраните в переменную sigma.
3. Зададим значение переменной target_prob, равное 0.90. Эта переменная будет использоваться для последующего вычисления критической дистанции.
4. Создадим объект нормального распределения distance_norm с заданными значениями mu и sigma.
5. Применим к созданному нормальному распределению distance_norm метод ppf() и в качестве аргумента передадим целевую вероятность target_prob. 

In [ ]:
# Вычисляем среднее значение
mu = df['distance'].mean()

# Вычисляем стандартное отклонение
sigma = df['distance'].std()

# Вероятность, для которой хотим найти значение (90% случаев)
target_prob = 0.9

# Создаём объект нормального распределения
distance_norm = st.norm(mu,sigma)

# Рассчитываем критическую дистанцию для заданного процентиля поездок
critical_distance = distance_norm.ppf(target_prob)

display(f'{100 * target_prob} % поездок имеют дистанцию ниже критического значения {critical_distance:.2f} М.')

## Итоговый вывод

В проекте проделана следующая работа:

1. Данные загружены в датафреймы: df_users_go, df_rides_go, df_subscriptions_go 
2. На этапе предобработки в датафрейме df_rides_go:
    - столбец 'date' приведен к типу 'datetime64';
    - создан новый столбец 'month', содержащий номер месяца;
    - время поездки столбец 'duration' округлен до целого числа и приведен к типу 'int64'.
3. В датафрейме пользователей df_users_go удалены дубликаты.
4. Разведочный анализ данных ответил на следующие вопросы бизнеса:
   - количество пользователей по городам;
   - количество пользователей для каждого типа подписки;
   - распределение подписчиков по возрастам;
   - доля несовершеннолетних пользователей - 5%;
   - средняя длительность поездки - 18 минут;
   - основная часть поездок занимает от 14 до 22 минут;
   - средняя длительность поездки для пользователя без подписки - 17 минут, с подпиской - 19 минут;
   - посчитана выручка на каждого пользователя;
   - найден пользователь с максимальной суммарной выручкой за весь период наблюдения.
5. Дальнейший анализ и проверка гипотез помогли выявить ключевые характеристики распределения данных и определить критические значения для принятия эффективных бизнес-решений:
   - средняя длительность поездки - 18,5 минут, стандартное отклонение - 6 минут;
   - вероятность поездки более 30 минут - 0,02;
   - вероятность того, что пользователь совершит поездку от 20 до 30 минут - 0,377;
   - 90% поездок - на дистанции ниже критического для технического состояния самоката.